<a href="https://colab.research.google.com/github/joaoBernardinoo/avoid-obstacle-collision/blob/main/controllers/cnn/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import torch
import torch.nn as nn
import sys
import os
import h5py
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
import matplotlib

from models import LidarVisionFusionNet

In [28]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
# @title Drive
if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')
  project_root = os.path.abspath(os.path.join(os.getcwd()))
  if project_root not in sys.path:
      sys.path.insert(0, project_root)
else:
  project_root = os.path.abspath(os.path.join(os.getcwd()))
  if project_root not in sys.path:
      sys.path.insert(0, project_root)

In [48]:

# matplotlib.use("notebook")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Configuration ---_
import sys

# Check if running in Colab
if 'google.colab' in sys.modules:
    SCRIPT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/cnn")
    DATASET_PATH = SCRIPT_DIR / 'merged_dataset.h5'
    BATCH_SIZE = 256
    print(f"Running in Google Colab. SCRIPT_DIR set to: {SCRIPT_DIR}, DATASET_PATH set to: {DATASET_PATH}, BATCH_SIZE set to: {BATCH_SIZE}")
else:
    SCRIPT_DIR = Path.cwd()
    DATASET_PATH = SCRIPT_DIR / 'cnn_dataset.h5'
    BATCH_SIZE = 64
    print(f"Running in local environment (VS Code?). SCRIPT_DIR set to: {SCRIPT_DIR}, DATASET_PATH set to: {DATASET_PATH}, BATCH_SIZE set to: {BATCH_SIZE}")

# Diretório para salvar os checkpoints
CHECKPOINT_DIR = SCRIPT_DIR / 'checkpoints'
PLOT_SAVE_PATH = SCRIPT_DIR / 'training_history.png'

# Cria o diretório de checkpoints se ele não existir
CHECKPOINT_DIR.mkdir(exist_ok=True)

# Image dimensions
IMG_HEIGHT = 40
IMG_WIDTH = 200
IMG_CHANNELS = 4  # BGRA from Webots

# Training parameters
EPOCHS = 40
LEARNING_RATE = 1e-4
N_SPLITS = 5  # Número de folds para K-Fold

DIST_SCALE = 3.14
ANGLE_SCALE = np.pi
LIDAR_MAX_VAL = 3.14

Using device: cuda
Running in local environment (VS Code?). SCRIPT_DIR set to: /home/dino/Documents/ia/controllers/cnn, DATASET_PATH set to: /home/dino/Documents/ia/controllers/cnn/cnn_dataset.h5, BATCH_SIZE set to: 64


In [19]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.amp.autocast_mode import autocast
from torch.amp.grad_scaler import GradScaler
from torch.utils.data import Dataset, DataLoader
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights


class DummyObstacleDataset(Dataset):
    def __init__(self, num_samples=1000, lidar_points=360, transform=None):
        self.num_samples = num_samples
        self.lidar_points = lidar_points
        self.transform = transform

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image = torch.randn(3, 224, 224)
        lidar = torch.randn(self.lidar_points)
        labels = torch.randn(2)

        if self.transform:
            pass

        return image, lidar, labels

if __name__ == '__main__':
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    LEARNING_RATE = 1e-4
    BATCH_SIZE = 32
    NUM_EPOCHS = 50
    LIDAR_POINTS = 20

    use_amp = DEVICE == 'cuda'
    print(f"Usando dispositivo: {DEVICE}")
    print(f"Treinamento com Precisão Mista (AMP): {'Ativado' if use_amp else 'Desativado'}")

    model = LidarVisionFusionNet(lidar_input_features=LIDAR_POINTS).to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)

    scaler = GradScaler("cuda",enabled=use_amp)

    data_transform = model.vision_transforms

    train_dataset = DummyObstacleDataset(num_samples=1000, lidar_points=LIDAR_POINTS, transform=data_transform)
    val_dataset = DummyObstacleDataset(num_samples=200, lidar_points=LIDAR_POINTS, transform=data_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    for epoch in range(NUM_EPOCHS):
        model.train()
        total_train_loss = 0

        for images, lidars, labels in train_loader:
            images = images.to(DEVICE)
            lidars = lidars.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()

            with autocast("cuda",enabled=use_amp):
                outputs = model(images, lidars)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, lidars, labels in val_loader:
                images = images.to(DEVICE)
                lidars = lidars.to(DEVICE)
                labels = labels.to(DEVICE)

                with autocast("cuda",enabled=use_amp):
                    outputs = model(images, lidars)
                    loss = criterion(outputs, labels)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss Treino: {avg_train_loss:.4f}, Loss Validação: {avg_val_loss:.4f}")

        scheduler.step(avg_val_loss)

Usando dispositivo: cuda
Treinamento com Precisão Mista (AMP): Ativado
Epoch [1/50], Loss Treino: 1.2230, Loss Validação: 1.1016


KeyboardInterrupt: 

In [9]:
def load_data(path: Path, max_samples=None):
    """
    Carrega o dataset de um único arquivo HDF5.
    """
    if not path.exists() or not path.is_file():
        print(f"Error: HDF5 file not found at '{path}'")
        raise FileNotFoundError

    with h5py.File(path, 'r') as hf:
        if 'camera_image' not in hf:
            print(f"Error: 'camera_image' dataset not found in '{path}'")
            raise ValueError

        print(f"Loading samples from '{path}'...")
        X_cam = np.array(hf['camera_image'], dtype=np.float32)
        X_lidar = np.array(hf['lidar_data'], dtype=np.float32)
        dist = np.array(hf['dist'], dtype=np.float32)
        angle = np.array(hf['angle'], dtype=np.float32)
        y = np.stack((dist, angle), axis=1)

        if max_samples:
            X_cam = X_cam[:max_samples]
            X_lidar = X_lidar[:max_samples]
            y = y[:max_samples]

    # Normaliza imagens
    X_cam /= 255.0


    X_lidar[np.isinf(X_lidar)] = LIDAR_MAX_VAL
    X_lidar /= LIDAR_MAX_VAL

    # --- NORMALIZE TARGETS (y) ---_
    y[:, 0] /= DIST_SCALE  # Normalize distance
    y[:, 1] /= ANGLE_SCALE   # Normalize angle
    y = np.clip(y, -1.0, 1.0)

    # Permuta os eixos da imagem para o formato do PyTorch (N, C, H, W)
    X_cam = np.transpose(X_cam, (0, 3, 1, 2))

    return (X_cam, X_lidar), y

In [11]:
# @title CNNNavigationModel
class CNNNavigationModel(nn.Module):
    """
    Define a arquitetura de um modelo CNN robusto para navegação usando dados de câmera e LiDAR.

    Este modelo aprimorado calcula dinamicamente o tamanho da entrada para as camadas
    totalmente conectadas, tornando-o robusto a mudanças nas dimensões da imagem de entrada
    ou na arquitetura da CNN. Ele também refatora a ramificação da câmera para maior clareza
    e inclui inicialização de pesos Kaiming.
    """
    def __init__(self, lidar_shape_in, img_shape=(IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH)):
        """
        Inicializa o modelo CNN.

        Args:
            lidar_shape_in (int): O número de características de entrada para a ramificação LiDAR.
            img_shape (tuple): A forma da imagem de entrada (C, H, W).
        """
        super(CNNNavigationModel, self).__init__()

        # --- Ramificação da Câmera ---_
        # Definimos a parte convolucional primeiro para calcular dinamicamente seu tamanho de saída.
        camera_conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=img_shape[0], out_channels=32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
        )

        # **MELHORIA 1: Cálculo dinâmico do tamanho das características**
        # Criamos um tensor "dummy" e o passamos pelas camadas convolucionais para descobrir o tamanho achatado.
        with torch.no_grad():
            dummy_input = torch.zeros(1, *img_shape)
            flattened_cam_size = camera_conv_layers(dummy_input).shape[1]


        # **MELHORIA 2: Clareza arquitetural**
        # Combinamos todo o pipeline de processamento da câmera em um único módulo sequencial.
        self.camera_branch = nn.Sequential(
            camera_conv_layers,
            nn.Linear(flattened_cam_size, 64),
            nn.ReLU()
        )

        # --- Ramificação do LiDAR ---_
        self.lidar_branch = nn.Sequential(
            nn.Linear(lidar_shape_in, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU()
        )

        # --- Cabeça Combinada ---_
        # Esta parte permanece conceitualmente a mesma.
        self.combined_head = nn.Sequential(
            nn.Linear(64 + 64, 128),  # 64 da câmera + 64 do LiDAR
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2),  # Saída de 2 valores (dist, angulo)
            nn.Tanh()          # Escala a saída para [-1, 1] para alvos normalizados
        )

        # **MELHORIA 3: Inicialização de Pesos**
        self.apply(self._init_weights)

    def _init_weights(self, module):
        """
        Aplica a inicialização Kaiming He às camadas Conv2d e Linear.
        """
        if isinstance(module, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_normal_(module.weight, mode='fan_in', nonlinearity='relu')
            if module.bias is not None:
                nn.init.constant_(module.bias, 0)

    def forward(self, cam_input, lidar_input):
        """
        Define a passagem para a frente (forward pass) do modelo.

        Args:
            cam_input (torch.Tensor): O tensor de entrada da câmera.
                                      Forma: (N, C, H, W)
            lidar_input (torch.Tensor): O tensor de entrada do LiDAR.
                                        Forma: (N, lidar_shape_in)

        Returns:
            torch.Tensor: O tensor de saída do modelo. Forma: (N, 2)
        """
        cam_features = self.camera_branch(cam_input)
        lidar_features = self.lidar_branch(lidar_input)
        combined_features = torch.cat((cam_features, lidar_features), dim=1)
        output = self.combined_head(combined_features)
        return output

In [ ]:
# @title plot

def plot_history(history):
    """Plota o histórico de treinamento e validação e salva em um arquivo."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.plot(history['loss'], label='Training Loss')
    ax1.plot(history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_ylabel('Loss (MSE)')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    ax2.plot(history['mae'], label='Training MAE')
    ax2.plot(history['val_mae'], label='Validation MAE')
    ax2.set_title('Model MAE')
    ax2.set_ylabel('MAE')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    plt.tight_layout()
    plt.savefig(PLOT_SAVE_PATH)  # Salva a figura em um arquivo
    plt.close(fig)  # Fecha a figura para liberar memória
    print(f"Training plot saved to '{PLOT_SAVE_PATH}'")
    plt.close(fig)  # Fecha a figura para liberar memória

In [ ]:
(X_cam, X_lidar), y = load_data(DATASET_PATH)

Loading samples from '/content/drive/MyDrive/Colab Notebooks/cnn/merged_dataset.h5'...


In [ ]:
(X_cam_test, X_lidar_test), y_test = load_data(SCRIPT_DIR / "test.h5")  # /home/dino/Documents/ia/controllers/cnn/test.h5

Loading samples from '/content/drive/MyDrive/Colab Notebooks/cnn/test.h5'...


In [ ]:
# prompt: /content/drive/MyDrive/Colab Notebooks/cnn/checkpoints
# I want to delete all files in this folder
if False:
    import shutil

    # Define the directory to clean
    directory_to_clean = SCRIPT_DIR / 'checkpoints'

    # Check if the directory exists
    if directory_to_clean.exists() and directory_to_clean.is_dir():
        print(f"Cleaning directory: {directory_to_clean}")
        # Remove all files and subdirectories within the directory
        for item in directory_to_clean.iterdir():
            if item.is_file():
                item.unlink()  # Delete the file
                print(f"Deleted file: {item}")
            elif item.is_dir():
                shutil.rmtree(item) # Delete the subdirectory and its contents
                print(f"Deleted directory: {item}")
        print("Cleaning complete.")
    else:
        print(f"Directory not found or is not a directory: {directory_to_clean}")

Cleaning directory: /content/drive/MyDrive/Colab Notebooks/cnn/checkpoints
Cleaning complete.


In [ ]:
# @title train_cnn.py


if X_cam is None or X_cam.shape[0] == 0:
    print("Exiting: No data loaded.")
else:
    print(f"Data loaded. Shapes: Cam={X_cam.shape}, Lidar={X_lidar.shape}, Target={y.shape}")

    kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    fold_histories = []

    best_overall_val_loss = float('inf')
    best_overall_checkpoint_path = None

    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_cam)):
        print(f"\n--- Starting Fold {fold+1}/{N_SPLITS} ---")

        # Define o caminho do checkpoint para este fold específico
        checkpoint_path = CHECKPOINT_DIR / f'checkpoint_fold_{fold+1}.pth'

        # Split data for this fold
        X_cam_train, y_train = X_cam[train_idx], y[train_idx]
        X_lidar_train = X_lidar[train_idx]

        X_cam_val, y_val = X_cam[val_idx], y[val_idx]
        X_lidar_val = X_lidar[val_idx]

        # Converte dados para tensores PyTorch
        train_dataset = TensorDataset(torch.tensor(
            X_cam_train), torch.tensor(X_lidar_train), torch.tensor(y_train))
        train_loader = DataLoader(
            train_dataset, batch_size=BATCH_SIZE, shuffle=True)

        val_dataset = TensorDataset(torch.tensor(
            X_cam_val), torch.tensor(X_lidar_val), torch.tensor(y_val))
        val_loader = DataLoader(
            val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Instancia o modelo e o otimizador
        model = CNNNavigationModel(lidar_shape_in=X_lidar.shape[1]).to(device)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Inicializa variáveis para o loop de treino e checkpoint
        start_epoch = 0
        best_val_loss = float('inf')
        history = {'loss': [], 'val_loss': [], 'mae': [],
                   'val_mae': [], "best_val_mae": 0.0}

        # --- Early Stopping parameters ---
        patience = 15  # Number of epochs to wait for improvement
        epochs_no_improve = 0
        min_delta = 0.002 # Minimum change to qualify as an improvement

        # --- LÓGICA PARA CARREGAR CHECKPOINT ---_
        if checkpoint_path.exists():
            print(f"Resuming training from checkpoint: {checkpoint_path}")
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch'] + 1
            best_val_loss = checkpoint['best_val_loss']
            history = checkpoint['history']
            # Reset early stopping counter if resuming
            epochs_no_improve = 0
            print(
                f"Resumed from epoch {start_epoch}. Best validation loss so far: {best_val_loss:.4f}")
        else:
            print(
                f"No checkpoint found for fold {fold+1}. Starting from scratch.")

        # Define as funções de perda
        criterion = nn.MSELoss()
        mae_criterion = nn.L1Loss()

        print(
            f"\n--- Starting Model Training for Fold {fold+1} from Epoch {start_epoch+1} ---")
        for epoch in range(start_epoch, EPOCHS):
            # --- Fase de Treinamento ---_
            model.train()
            running_loss, running_mae = 0.0, 0.0
            progress_bar = tqdm(
                train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{EPOCHS} [T]", leave=False)
            for cam_batch, lidar_batch, target_batch in progress_bar:
                cam_batch, lidar_batch, target_batch = cam_batch.to(
                    device), lidar_batch.to(device), target_batch.to(device)
                optimizer.zero_grad()
                outputs = model(cam_batch, lidar_batch)
                loss = criterion(outputs, target_batch)
                mae = mae_criterion(outputs, target_batch)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                running_mae += mae.item()
                progress_bar.set_postfix(loss=loss.item(), mae=mae.item())

            epoch_loss = running_loss / len(train_loader)
            epoch_mae = running_mae / len(train_loader)
            history['loss'].append(epoch_loss)
            history['mae'].append(epoch_mae)

            # --- Fase de Validação ---_
            model.eval()
            val_loss, val_mae = 0.0, 0.0
            with torch.no_grad():
                for cam_batch, lidar_batch, target_batch in val_loader:
                    cam_batch, lidar_batch, target_batch = cam_batch.to(
                        device), lidar_batch.to(device), target_batch.to(device)
                    outputs = model(cam_batch, lidar_batch)
                    val_loss += criterion(outputs, target_batch).item()
                    val_mae += mae_criterion(outputs, target_batch).item()

            epoch_val_loss = val_loss / len(val_loader)
            epoch_val_mae = val_mae / len(val_loader)
            history['val_loss'].append(epoch_val_loss)
            history['val_mae'].append(epoch_val_mae)

            print(
                f"Fold {fold+1}/{N_SPLITS} Epoch {epoch+1}/{EPOCHS} - "
                f"Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f} - "
                f"Val Loss: {epoch_val_loss:.4f}, Val MAE: {epoch_val_mae:.4f}"
            )

            if epoch_val_loss < best_val_loss - min_delta:
                best_val_loss = epoch_val_loss
                epochs_no_improve = 0
                print(
                    f"  -> New best validation loss: {best_val_loss:.4f}. Saving checkpoint...")
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_val_loss': best_val_loss,
                    'best_val_mae': epoch_val_mae,
                    'history': history
                }, checkpoint_path)

                history['best_val_mae'] = epoch_val_mae # Update best MAE when a new best loss is found
            else:
                epochs_no_improve += 1
                print(f"  -> Validation loss did not improve by at least {min_delta}. Epochs without improvement: {epochs_no_improve}")

            if epochs_no_improve == patience:
                print(f"  -> Early stopping triggered after {patience} epochs without improvement.")
                break # Exit the epoch loop


        # If training stopped due to early stopping, load the best model state for this fold
        if epochs_no_improve == patience and checkpoint_path.exists():
             print(f"  -> Loading best model state for fold {fold+1} after early stopping.")
             checkpoint = torch.load(checkpoint_path, map_location=device)
             model.load_state_dict(checkpoint['model_state_dict'])
             # Update history with the best validation MAE from the saved checkpoint
             history['best_val_mae'] = checkpoint['best_val_mae']


        fold_histories.append(history)

        # Update best overall checkpoint if this fold performed better
        if checkpoint_path.exists():
            checkpoint = torch.load(checkpoint_path, map_location=device)
            current_fold_best_val_loss = checkpoint['best_val_loss']
            if current_fold_best_val_loss < best_overall_val_loss:
                best_overall_val_loss = current_fold_best_val_loss
                best_overall_checkpoint_path = checkpoint_path

    # Agrega e plota os resultados de todos os folds
    if fold_histories:
        # Ensure all histories have the same length for averaging (due to early stopping)
        min_len = min(len(h['loss']) for h in fold_histories)
        for h in fold_histories:
            for key in h:
                if isinstance(h[key], list):
                    h[key] = h[key][:min_len]


        avg_history = {
            'loss': np.mean([h['loss'] for h in fold_histories], axis=0).tolist(),
            'val_loss': np.mean([h['val_loss'] for h in fold_histories], axis=0).tolist(),
            'mae': np.mean([h['mae'] for h in fold_histories], axis=0).tolist(),
            'val_mae': np.mean([h['val_mae'] for h in fold_histories], axis=0).tolist(),
            # best_val_mae is a single value per fold, so we average the last recorded best
            'best_val_mae': np.mean([h['best_val_mae'] for h in fold_histories]).tolist()
        }
        plot_history(avg_history)

    print("\nTraining complete.")

Data loaded. Shapes: Cam=(5891, 4, 40, 200), Lidar=(5891, 20), Target=(5891, 2)

--- Starting Fold 1/5 ---
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
No checkpoint found for fold 1. Starting from scratch.

--- Starting Model Training for Fold 1 from Epoch 1 ---


Fold 1/5 Epoch 1/40 - Loss: 0.3279, MAE: 0.4756 - Val Loss: 0.0599, Val MAE: 0.1905
  -> New best validation loss: 0.0599. Saving checkpoint...


Fold 1/5 Epoch 2/40 - Loss: 0.1639, MAE: 0.3297 - Val Loss: 0.0625, Val MAE: 0.2031
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 1


Fold 1/5 Epoch 3/40 - Loss: 0.1230, MAE: 0.2832 - Val Loss: 0.0263, Val MAE: 0.1202
  -> New best validation loss: 0.0263. Saving checkpoint...


Fold 1/5 Epoch 4/40 - Loss: 0.0989, MAE: 0.2513 - Val Loss: 0.0297, Val MAE: 0.1356
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 1


...

Fold 5/5 Epoch 36/40 - Loss: 0.0108, MAE: 0.0773 - Val Loss: 0.0058, Val MAE: 0.0497
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 9


Fold 5/5 Epoch 37/40 - Loss: 0.0104, MAE: 0.0759 - Val Loss: 0.0057, Val MAE: 0.0492
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 10


Fold 5/5 Epoch 38/40 - Loss: 0.0102, MAE: 0.0743 - Val Loss: 0.0058, Val MAE: 0.0511
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 11


Fold 5/5 Epoch 39/40 - Loss: 0.0099, MAE: 0.0738 - Val Loss: 0.0060, Val MAE: 0.0533
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 12


Fold 5/5 Epoch 40/40 - Loss: 0.0098, MAE: 0.0734 - Val Loss: 0.0058, Val MAE: 0.0501
  -> Validation loss did not improve by at least 0.002. Epochs without improvement: 13
Training plot saved to '/content/drive/MyDrive/Colab Notebooks/cnn/training_history.png'

Training complete.


In [ ]:
final_model_path = SCRIPT_DIR / 'final_cnn_model.pth'


In [ ]:
def fine_tune_model(best_checkpoint_path: Path, full_dataset_path: Path, fine_tune_epochs: int = 2, fine_tune_lr: float = 1e-3):
    print(f"\n--- Starting Fine-tuning on Entire Dataset ---")
    print(f"Loading best model from: {best_checkpoint_path}")

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load the full dataset
    (X_cam, X_lidar), y = load_data(full_dataset_path)
    if X_cam is None or X_cam.shape[0] == 0:
        print("Exiting fine-tuning: No data loaded for full dataset.")
        return

    print(
        f"Full dataset loaded. Shapes: Cam={X_cam.shape}, Lidar={X_lidar.shape}, Target={y.shape}")

    # Create DataLoader for the full dataset
    full_dataset = TensorDataset(torch.tensor(
        X_cam), torch.tensor(X_lidar), torch.tensor(y))
    full_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Instantiate model and load state dict
    model = CNNNavigationModel(lidar_shape_in=X_lidar.shape[1]).to(device)
    checkpoint = torch.load(best_checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Optimizer for fine-tuning (can be a new one with lower LR)
    optimizer = optim.Adam(model.parameters(), lr=fine_tune_lr)
    # Optionally load optimizer state if resuming fine-tuning
    if 'optimizer_state_dict' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    criterion = nn.MSELoss()
    mae_criterion = nn.L1Loss()

    print(f"Fine-tuning for {fine_tune_epochs} epochs with LR: {fine_tune_lr}")

    model.train()  # Set model to training mode
    for epoch in range(fine_tune_epochs):
        running_loss, running_mae = 0.0, 0.0
        progress_bar = tqdm(
            full_loader, desc=f"Fine-tune Epoch {epoch+1}/{fine_tune_epochs}", leave=False)
        for cam_batch, lidar_batch, target_batch in progress_bar:
            cam_batch, lidar_batch, target_batch = cam_batch.to(
                device), lidar_batch.to(device), target_batch.to(device)
            optimizer.zero_grad()
            outputs = model(cam_batch, lidar_batch)
            loss = criterion(outputs, target_batch)
            mae = mae_criterion(outputs, target_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_mae += mae.item()
            progress_bar.set_postfix(loss=loss.item(), mae=mae.item())

        epoch_loss = running_loss / len(full_loader)
        epoch_mae = running_mae / len(full_loader)
        print(
            f"Fine-tune Epoch {epoch+1}/{fine_tune_epochs} - Loss: {epoch_loss:.4f}, MAE: {epoch_mae:.4f}")

    # Save the final fine-tuned model
    torch.save(model.state_dict(), FINAL_MODEL_PATH)
    print(f"Fine-tuned model saved to: {FINAL_MODEL_PATH}")
best_overall_checkpoint_path = True
if best_overall_checkpoint_path:
    fine_tune_model(Path("/content/drive/MyDrive/Colab Notebooks/cnn/checkpoints/checkpoint_fold_2.pth"), DATASET_PATH)
else:
    print("\nNo best model found for fine-tuning.")


--- Starting Fine-tuning on Entire Dataset ---
Loading best model from: /content/drive/MyDrive/Colab Notebooks/cnn/checkpoints/checkpoint_fold_2.pth
Loading samples from '/content/drive/MyDrive/Colab Notebooks/cnn/merged_dataset.h5'...
Full dataset loaded. Shapes: Cam=(5891, 4, 40, 200), Lidar=(5891, 20), Target=(5891, 2)
Tamanho da característica da câmera achatada calculado dinamicamente: 24576
Fine-tuning for 2 epochs with LR: 0.001


Fine-tune Epoch 1/2 - Loss: 0.0130, MAE: 0.0847


Fine-tune Epoch 2/2 - Loss: 0.0144, MAE: 0.0891
Fine-tuned model saved to: /content/drive/MyDrive/Colab Notebooks/cnn/final_cnn_model.pth


In [ ]:
import h5py
import numpy as np
from pathlib import Path

# Define the paths to the input and output HDF5 files
# Adjust these paths based on your environment if necessary
if 'google.colab' in sys.modules:
    SCRIPT_DIR = Path("/content/drive/MyDrive/Colab Notebooks/cnn")
else:
    SCRIPT_DIR = Path.cwd()

input_file_1 = SCRIPT_DIR / 'cnn_dataset.h5'
input_file_2 = SCRIPT_DIR / 'test.h5'
output_file = SCRIPT_DIR / 'merged_dataset.h5'

print(f"Merging {input_file_1} and {input_file_2} into {output_file}")

try:
    # Open the input files in read mode
    with h5py.File(input_file_1, 'r') as hf1, \
         h5py.File(input_file_2, 'r') as hf2, \
         h5py.File(output_file, 'w') as hf_out:

        # Assume both files have the same datasets, get dataset names from the first file
        dataset_names = list(hf1.keys())

        for name in dataset_names:
            print(f"Merging dataset: {name}")
            # Read data from both input files
            data1 = np.array(hf1[name])
            data2 = np.array(hf2[name])

            # Concatenate the data along the first axis
            merged_data = np.concatenate((data1, data2), axis=0)

            # Write the merged data to the output file
            hf_out.create_dataset(name, data=merged_data)

    print("Merging complete.")

except FileNotFoundError as e:
    print(f"Error: {e}. Please ensure the input files exist at the specified paths.")
except Exception as e:
    print(f"An error occurred during merging: {e}")

Merging /content/drive/MyDrive/Colab Notebooks/cnn/cnn_dataset.h5 and /content/drive/MyDrive/Colab Notebooks/cnn/test.h5 into /content/drive/MyDrive/Colab Notebooks/cnn/merged_dataset.h5
Merging dataset: angle
Merging dataset: camera_image
Merging dataset: dist
Merging dataset: lidar_data
Merging complete.
